In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/31 11:37:30 WARN Utils: Your hostname, DavidNg-8242-NB resolves to a loopback address: 127.0.1.1; using 172.24.225.125 instead (on interface eth0)
23/03/31 11:37:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/31 11:37:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [48]:
year = 2022

for month in range(3, 13, 3):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/china/hk/hong-kong/{year}/{month:02d}/'
    output_path = f'data/pq/china/hk/hong-kong/{year}/{month:02d}/'

    df_hk = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("multiline", "true") \
        .option("escape", "\"") \
        .csv(input_path)

    df_hk \
        .repartition(1) \
        .write.parquet(output_path, mode='overwrite')

processing data for 2022/3


processing data for 2022/6


processing data for 2022/9
processing data for 2022/12


In [49]:
df = spark.read.parquet('data/pq/china/hk/hong-kong/2022/12/')

In [51]:
df.columns

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'source',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30

In [52]:
df.createOrReplaceTempView('hk_data')

In [66]:
spark.sql("""
SELECT
availability_30,
availability_60,
availability_90,
availability_365,
neighbourhood_cleansed,
property_type,
room_type,
accommodates,
price
FROM
    hk_data
WHERE
    availability_30>0
""").show()

+---------------+---------------+---------------+----------------+----------------------+--------------------+---------------+------------+---------+
|availability_30|availability_60|availability_90|availability_365|neighbourhood_cleansed|       property_type|      room_type|accommodates|    price|
+---------------+---------------+---------------+----------------+----------------------+--------------------+---------------+------------+---------+
|             29|             59|             89|             364|     Central & Western|        Entire condo|Entire home/apt|           2|  $709.00|
|             23|             53|             79|              79|     Central & Western|  Entire rental unit|Entire home/apt|           3|  $546.00|
|             30|             60|             90|             365|         Yau Tsim Mong|Shared room in ho...|    Shared room|           1|  $180.00|
|             30|             60|             90|             365|     Central & Western|Private roo